# Моя реализация алгоритма KNN для бинарной классификации

Импортируем необходимые библиотеки для создания класса и для сравнения результатов предсказания с встроенной в sklearn моделью.

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier

## Данная модель принимает на вход следующие параметры:

k - заданное число ближайших соседей, на основании которых происходит классификация. По умолчанию: 3

metric - метрика, использующаяся для расчета расстояния между объектами. Доступные значения: 'euclidean', 'manhattan', 'chebyshev', 'cosine' - Евклидова, Манхеттена, Чебышева и косинусная соответственно. По умолчанию: 'euclidean'

weight - система взвешивания ближайших соседей. Доступные вариации: 'uniform', 'rank', 'distance' - равномерное, ранговое и по расстоянию соответственно. По умолчанию: uniform

## MyKNNClf поддерживает следующие методы:

fit(X: pd.DataFrame(), y: pd.Series()) - обучение модели, где X - признаки, а y - классы объектов

predict(X) - предсказывает значения классов по признакам

prefict_proba(X) - предсказывает вероятности того, что объект относится к классу 1


In [14]:
class MyKNNClf:
  def __init__(self, k=3, metric = 'euclidean', weight='uniform'):
    self.k = k
    self.train_size = (0, 0)
    self.metric = metric
    self.weight = weight

  def __str__(self):
    return f'MyKNNClf class: k={self.k}'

  def fit(self, X: pd.DataFrame(), y: pd.Series()):
    self.X = np.array(X)
    self.y = np.array(y)
    self.train_size = X.shape

  def predict(self, X: pd.DataFrame):
    y_train = np.array(self.y)#[:, np.newaxis]
    X_train = self.X

    if self.metric == 'euclidean':
      distances = np.sum((np.array(X_train) - np.array(X)[:, np.newaxis])**2, axis=2)**0.5
    elif self.metric == 'manhattan':
      distances = np.sum(abs(np.array(X_train) - np.array(X)[:, np.newaxis]), axis=2)
    elif self.metric == 'chebyshev':
      distances = np.max(abs(np.array(X_train) - np.array(X)[:, np.newaxis]), axis=2)
    elif self.metric == 'cosine':
      distances = 1 - np.sum(np.array(X_train)*np.array(X)[:, np.newaxis], axis=2)/np.sum((np.array(X_train))**2, axis=1)**0.5/np.sum((np.array(X))**2, axis=1)[:, np.newaxis]**0.5

    y_pred = []
    if self.weight == 'uniform':
      class_indexes = np.argsort(distances, axis=1)[:, :self.k]
      k_nearest_classes = np.squeeze(y_train[class_indexes])
      y_pred = []
      for i in range(len(class_indexes)):
            y_pred.append(int(np.mean(k_nearest_classes[i])>=0.5))
      y_pred = np.array(y_pred)

    elif self.weight == 'rank':
      class_indexes = np.argsort(distances, axis=1)[:, :self.k] #индексы ближайших соседей по возрастанию, фактически мы можем просто каждому из этих индексов по порядку присвоить номер от 1 до n
      nearest_classes = (y_train[class_indexes]==1)
      self.nearest_classes = nearest_classes
      ranks = np.ones(class_indexes.shape)+np.arange(class_indexes.shape[1])
      self.ranks = ranks
      for i in range(len(nearest_classes)):
        y_pred.append(np.sum(1/ranks[i][np.squeeze(nearest_classes)[i]])/np.sum(1/np.arange(1, nearest_classes.shape[1]+1)))
      y_pred = (np.array(y_pred)>=0.5).astype(int)

    elif self.weight == 'distance':
      class_indexes = np.argsort(distances, axis=1)[:, :self.k] # индексы ближайших k соседей к каждому из 250 элементов
      y_pred = []

      for i in range(len(class_indexes)):
        dist = distances[i]
        indexes = class_indexes[i]
        weights = 1/(dist[indexes]+1e-6) #веса для каждого из k соседей, расположенных в порядке возрастания
        y_pred.append(np.sum(weights*y_train[indexes])/np.sum(weights)) # каждый из этих соседей равен 0 либо 1. Если 1, то вес будет учтен при умножении, иначе нет.
      y_pred = (np.array(y_pred)>=0.5).astype(int)


    return y_pred

  def predict_proba(self, X: pd.DataFrame):
    y_train = np.array(self.y)#[:, np.newaxis]
    X_train = self.X
    y_pred = []

    if self.metric == 'euclidean':
      distances = np.sum((np.array(X_train) - np.array(X)[:, np.newaxis])**2, axis=2)**0.5
    elif self.metric == 'manhattan':
      distances = np.sum(abs(np.array(X_train) - np.array(X)[:, np.newaxis]), axis=2)
    elif self.metric == 'chebyshev':
      distances = np.max(abs(np.array(X_train) - np.array(X)[:, np.newaxis]), axis=2)
    elif self.metric == 'cosine':
      distances = 1 - np.sum(np.array(X_train)*np.array(X)[:, np.newaxis], axis=2)/np.sum((np.array(X_train))**2, axis=1)**0.5/np.sum((np.array(X))**2, axis=1)[:, np.newaxis]**0.5

    if self.weight == 'uniform':
      class_indexes = np.argsort(distances, axis=1)[:, :self.k]
      k_nearest_classes = np.squeeze(y_train[class_indexes])
      for i in range(len(k_nearest_classes)):
            y_pred.append(np.mean(k_nearest_classes[i]))
      y_pred = np.array(y_pred)

    elif self.weight == 'rank':
      class_indexes = np.argsort(distances, axis=1)[:, :self.k] #индексы ближайших соседей по возрастанию, фактически мы можем просто каждому из этих индексов по порядку присвоить номер от 1 до n
      nearest_classes = (y_train[class_indexes]==1)
      ranks = np.ones(class_indexes.shape)+np.arange(class_indexes.shape[1])
      for i in range(len(nearest_classes)):
        y_pred.append(np.sum(1/ranks[i][np.squeeze(nearest_classes)[i]])/np.sum(1/np.arange(1, nearest_classes.shape[1]+1)))
      y_pred = np.array(y_pred)

    elif self.weight == 'distance':
      class_indexes = np.argsort(distances, axis=1)[:, :self.k] # индексы ближайших k соседей к каждому из 250 элементов
      y_pred = []
      for i in range(len(class_indexes)):
        dist = distances[i]
        indexes = class_indexes[i]
        weights = 1/(dist[indexes]+1e-6) #веса для каждого из k соседей, расположенных в порядке возрастания
        y_pred.append(np.sum(weights*y_train[indexes])/np.sum(weights)) # каждый из этих соседей равен 0 либо 1. Если 1, то вес будет учтен при умножении, иначе нет.
      y_pred = np.array(y_pred)

    return y_pred

Создаем выборку размера 10000 с помощью встроенной в sklearn функции

In [7]:
X, y = make_classification(n_samples=10000, n_features=10, n_informative=7, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)

X_train, X_test, y_train, y_test = train_test_split(X, y)

Обучим нашу модель и спрогнозируем классы для тех же данных. Затем обучим встроенную в sklearn модель и сравним их прогнозы.

In [20]:
model = MyKNNClf(k=3, metric = 'euclidean', weight='distance')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

np.mean(y_pred == y_test)

0.9116

In [21]:
model_sk = KNeighborsClassifier(3, weights='distance', metric='euclidean')
model_sk.fit(X_train, y_train)
y_pred_sk = model_sk.predict(X_test)

np.mean(y_pred_sk==y_test)

0.9116

In [22]:
(y_pred==y_pred_sk).sum()

2500

In [15]:
model = MyKNNClf(k=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

np.mean(y_pred == y_test)

0.9116

In [16]:
model_sk = KNeighborsClassifier(3)
model_sk.fit(X_train, y_train)
y_pred_sk = model_sk.predict(X_test)

np.mean(y_pred_sk==y_test)

0.9116

In [18]:
(y_pred==y_pred_sk).sum()

2498

Как мы видим, данная реализация дает практически идентичный встроенной модели результат, хотя и проигрывает ей по времени из-за некоторых не совсем оптимальных решений внутри кода. Поскольку модель показывает сопоставимую точность и при этом поддерживает несколько вариаций метрик и весов, я считаю свою реализацю успешной.